In [21]:
import pandas as pd
import os
from pathlib import Path
from scipy.stats import ranksums


In [22]:
NEURAL_NETWORKS = ['DenseNet201', 'MobileNetV2', 'InceptionV3', "ResNet50"]
COLUMNS = ["accuracy", "precision","sensitivity", "specificity", "f1_score", "auc", "npv", "mcc", "val_sensitivity", "val_accuracy", "val_precision", "val_specificity", "val_f1_score", "val_auc", "val_npv", "val_mcc"]
SUBSETS = ['Dataset01_100', 'Dataset01_95.0', 'Dataset01_90.0', 'Dataset01_85.0', 'Dataset01_80.0', 'Dataset01_75.0', 
           'Dataset01_70.0', 'Dataset01_65.0', 'Dataset01_60.0', 'Dataset01_55.0', 'Dataset01_50.0', 'Dataset01_45.0',
           'Dataset01_40.0', 'Dataset01_35.0', 'Dataset01_30.0', 'Dataset01_25.0', 'Dataset01_20.0', 'Dataset01_15.0', 
           'Dataset01_10.0', 'Dataset01_5.0']


In [23]:
path_project = Path().absolute().parent.parent.parent
results_path = path_project / "6_resultados"


In [24]:

result_ultimate = {}
for network in NEURAL_NETWORKS:
    output_file = results_path / 'analitycs' / 'by_train' / '_pre_plot' / f'{network}_result_for_analytic.csv'
    for subset in SUBSETS:
        # Path to the CSVs for training
        csvs_path = results_path / subset / network / "metrics" / "csvs" / "_refined"
        if not os.path.exists(csvs_path):
            continue
        print(csvs_path)
       
        # Read and concatenate training CSV files
        if os.path.isfile(output_file):
            result_df = pd.read_csv(output_file)
        else:
            result_df = pd.DataFrame()

        df = pd.read_csv(csvs_path / os.listdir(csvs_path)[0])
        

        runtime = df["runtime"] # second to minutes
        runtime = runtime.sum() / 60

        val_runtime = df["val_runtime"] # second to minutes
        val_runtime = val_runtime.sum() / 60
        
        total_runtime = runtime + val_runtime
        result_data = {"subset": int(float(subset.split("_")[-1])), "model": network,
                        "runtime": runtime, "val_runtime": val_runtime, "total_runtime": total_runtime}
        for column in COLUMNS:
            clean_column = df[column].dropna()
            # Calcular quartis e IQR
            if clean_column.empty:
                first_quartile, third_quartile = 0, 0
                column_median, column_mean = 0, 0
                iqr = 0
                lower_whisker, upper_whisker = 0, 0
            else:
                first_quartile, third_quartile = clean_column.quantile([0.25, 0.75])
                iqr = third_quartile - first_quartile
                column_median = clean_column.median()
                column_mean = clean_column.mean()

                # Calcular whiskers com verificação para IQR zero
                if iqr == 0:
                    lower_whisker = first_quartile
                    upper_whisker = third_quartile
                else:
                    lower_whisker = first_quartile - 1.5 * iqr
                    upper_whisker = third_quartile + 1.5 * iqr

            # Armazenar resultados no dicionário
            result_data[f"{column}_mean"] = 100 *  column_mean
            result_data[f"{column}_median"] = 100 *  column_median
            result_data[f"{column}_lower"] = first_quartile
            result_data[f"{column}_upper"] = third_quartile
            result_data[f"{column}_lower_whisker"] = lower_whisker
            result_data[f"{column}_upper_whisker"] = upper_whisker

        result_temp_df = pd.DataFrame([result_data])
        result_df = pd.concat(
            [result_df, result_temp_df], ignore_index=True)
        result_df = result_df.sort_values(by=["subset"])
        # result_df: pd.DataFrame = result_df.append(result_data, ignore_index = True)
        result_df.to_csv(output_file, index=None)

/media/williancaddd/CODES/projects/malaria-pibiti/6_resultados/Dataset01_100/DenseNet201/metrics/csvs/_refined
/media/williancaddd/CODES/projects/malaria-pibiti/6_resultados/Dataset01_95.0/DenseNet201/metrics/csvs/_refined
/media/williancaddd/CODES/projects/malaria-pibiti/6_resultados/Dataset01_90.0/DenseNet201/metrics/csvs/_refined
/media/williancaddd/CODES/projects/malaria-pibiti/6_resultados/Dataset01_85.0/DenseNet201/metrics/csvs/_refined
/media/williancaddd/CODES/projects/malaria-pibiti/6_resultados/Dataset01_80.0/DenseNet201/metrics/csvs/_refined
/media/williancaddd/CODES/projects/malaria-pibiti/6_resultados/Dataset01_75.0/DenseNet201/metrics/csvs/_refined
/media/williancaddd/CODES/projects/malaria-pibiti/6_resultados/Dataset01_70.0/DenseNet201/metrics/csvs/_refined
/media/williancaddd/CODES/projects/malaria-pibiti/6_resultados/Dataset01_65.0/DenseNet201/metrics/csvs/_refined
/media/williancaddd/CODES/projects/malaria-pibiti/6_resultados/Dataset01_60.0/DenseNet201/metrics/csvs/_r